In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib import request
import time

In [3]:
years = np.arange(2000,2020,1).tolist()

for year in years:
    web_name = 'https://www.basketball-reference.com/leagues/NBA_'+str(year)+'_per_game.html'
    website = request.urlopen(web_name).read()
    soup = BeautifulSoup(website,'lxml')

    table = soup.find('table', attrs={'class': 'sortable', 'id': 'per_game_stats'})
    table_headers = [header.text for header in table.find('thead').find_all('th')]
    table_rows = table.find_all('tr')

    final_data = []
    # For each table row,
    for tr in table_rows:
        # Make a list of the table data tags for this row
        td = tr.find_all('td')
        # Extract just the cell data and make a list
        row = [tr.text for tr in td]
        # Append the extracted data
        final_data.append(row)


    df = pd.DataFrame(final_data[1:], columns=table_headers[1:])
    df.to_csv('../data/nba_stats_'+str(year)+'.csv')
    
    time.sleep(1)